In [7]:
import numpy as np

import torch
from torch import nn
import torch.nn.functional as F


class MultiHeadAttention(nn.Module):
    """ Multi-Head Attention module """

    def __init__(self, n_head, d_model, d_k, d_v, dropout=0.1):
        super().__init__()

        self.n_head = n_head
        self.d_k = d_k
        self.d_v = d_v

        self.w_qs = nn.Linear(d_model, n_head * d_k)
        self.w_ks = nn.Linear(d_model, n_head * d_k)
        self.w_vs = nn.Linear(d_model, n_head * d_v)
        nn.init.normal_(self.w_qs.weight, mean=0, std=np.sqrt(2.0 / (d_model + d_k)))
        nn.init.normal_(self.w_ks.weight, mean=0, std=np.sqrt(2.0 / (d_model + d_k)))
        nn.init.normal_(self.w_vs.weight, mean=0, std=np.sqrt(2.0 / (d_model + d_v)))

        self.attention = ScaledDotProductAttention(temperature=np.power(d_k, 0.5))
        self.layer_norm = nn.LayerNorm(d_model)

        self.fc = nn.Linear(n_head * d_v, d_model)
        nn.init.xavier_normal_(self.fc.weight)

        self.dropout = nn.Dropout(dropout)

    def forward(self, q, k, v):

        d_k, d_v, n_head = self.d_k, self.d_v, self.n_head

        sz_b, len_q, _ = q.size()
        sz_b, len_k, _ = k.size()
        sz_b, len_v, _ = v.size()

        residual = q
        q = self.w_qs(q).view(sz_b, len_q, n_head, d_k)
        k = self.w_ks(k).view(sz_b, len_k, n_head, d_k)
        v = self.w_vs(v).view(sz_b, len_v, n_head, d_v)
        q = q.permute(2, 0, 1, 3).contiguous().view(-1, len_q, d_k)
        k = k.permute(2, 0, 1, 3).contiguous().view(-1, len_k, d_k)
        v = v.permute(2, 0, 1, 3).contiguous().view(-1, len_v, d_v)
        output, attn = self.attention(q, k, v)

        output = output.view(n_head, sz_b, len_q, d_v)
        output = output.permute(1, 2, 0, 3).contiguous().view(sz_b, len_q, -1)

        output = self.dropout(self.fc(output))
        output = self.layer_norm(output + residual)

        return output, attn


class PositionwiseFeedForward(nn.Module):
    """ A two-feed-forward-layer module """

    def __init__(self, d_in, d_hid, dropout=0.1):
        super().__init__()
        self.w_1 = nn.Conv1d(d_in, d_hid, 1)
        self.w_2 = nn.Conv1d(d_hid, d_in, 1)
        self.layer_norm = nn.LayerNorm(d_in)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        residual = x
        output = x.transpose(1, 2)
        output = self.w_2(F.relu(self.w_1(output)))
        output = output.transpose(1, 2)
        output = self.dropout(output)
        output = self.layer_norm(output + residual)
        return output


class ScaledDotProductAttention(nn.Module):
    """ Scaled Dot-Product Attention """

    def __init__(self, temperature, attn_dropout=0.1):
        super().__init__()
        self.temperature = temperature
        self.dropout = nn.Dropout(attn_dropout)
        self.softmax = nn.Softmax(dim=2)

    def forward(self, q, k, v):

        attn = torch.bmm(q, k.transpose(1, 2))
        attn = attn / self.temperature

        attn = self.softmax(attn)
        attn = self.dropout(attn)
        output = torch.bmm(attn, v)

        return output, attn


class EncoderLayer(nn.Module):
    """ Compose with two layers """

    def __init__(self, d_model, d_inner=1024, n_head=8, d_k=64, d_v=64, dropout=0.1):
        super(EncoderLayer, self).__init__()
        self.slf_attn = MultiHeadAttention(n_head, d_model, d_k, d_v, dropout=dropout)
        self.pos_ffn = PositionwiseFeedForward(d_model, d_inner, dropout=dropout)

    def forward(self, enc_input):
        enc_output, enc_slf_attn = self.slf_attn(enc_input, enc_input, enc_input)
        
        enc_output = self.pos_ffn(enc_output)

        return enc_output, enc_slf_attn


In [8]:
tens = torch.randn(100,10,14)
encode = EncoderLayer(14)

In [10]:
encode(tens)[0]

tensor([[[-1.0406, -1.2483,  0.6753,  ...,  0.1709,  1.5389,  1.2648],
         [ 0.9159, -2.2183, -1.2717,  ..., -0.9412,  1.7283, -0.2817],
         [-1.1294,  1.2231,  1.2109,  ..., -0.8877, -0.1374,  0.4508],
         ...,
         [-0.1608, -0.8997,  0.1449,  ..., -0.4128,  1.5438, -1.1066],
         [-0.0599,  0.4641, -1.9637,  ..., -0.3171, -1.6015,  1.0451],
         [ 0.1465,  0.8424, -0.5878,  ..., -0.3073,  1.2185, -0.3230]],

        [[ 0.0344, -0.5854, -0.1213,  ...,  0.7590, -1.0364,  1.3861],
         [ 0.7507, -0.2037,  0.8846,  ..., -1.0601,  0.9511,  1.7958],
         [-0.5900,  0.1403,  0.7025,  ...,  0.0994,  0.0293, -1.1251],
         ...,
         [ 0.1602, -1.3555,  0.8423,  ...,  1.5417,  0.3075, -0.8635],
         [-0.6263, -0.0900,  1.9712,  ..., -0.4856,  0.6477, -1.3915],
         [-1.9873, -0.2725, -0.2912,  ..., -0.4379, -0.5548, -1.1041]],

        [[-0.3450,  0.4920, -0.0444,  ..., -1.7171,  1.1074, -0.7276],
         [ 1.2170, -0.6998, -0.1333,  ..., -0